In [7]:
import pandas as pd
import numpy as np
import scipy.linalg as LA
from scipy import optimize
from funcs import linear_solve_cholesky, get_approx_error_banks, get_fitted_vals

In [8]:
df = pd.read_excel("financials2012.xlsx", index_col="Date")

Problem 3 (i)

In [9]:
weekly_ret = (df/df.shift(-1) - 1).dropna()
print("The weekly percent returns matrix is\n")
weekly_ret

The weekly percent returns matrix is



,JPM,GS,MS,BAC,RBS,CS,UBS,RY (RBC),BCS (Barclays)
Date,,,,,,,,,
2012-10-15,0.016819,0.028453,0.012709,0.035088,0.034722,0.047258,0.034893,0.016215,0.008796
2012-10-08,-0.002158,0.007460,-0.010857,-0.021459,0.021277,-0.006203,-0.014844,-0.012268,0.019310
2012-10-01,0.038079,0.049525,0.045400,0.055493,0.016827,0.067139,0.050903,0.022296,0.045422
2012-09-24,-0.009857,-0.026045,-0.019906,-0.030735,-0.066218,-0.075612,-0.057276,0.000000,-0.038141
2012-09-17,-0.016719,-0.038233,-0.063596,-0.046073,-0.011099,-0.009952,-0.041543,-0.006576,-0.026334
2012-09-10,0.057934,0.043239,0.067916,0.085227,0.146310,0.092155,0.091498,0.005568,0.124525
2012-09-04,0.058042,0.100359,0.138667,0.102757,0.093185,0.098650,0.107623,0.026067,0.132416
2012-08-27,-0.000813,0.011675,0.030220,-0.020859,0.014104,0.002603,-0.004464,0.033014,-0.019393
2012-08-20,0.005176,0.013088,-0.002056,0.020025,-0.027435,0.052603,0.014493,-0.006232,-0.017399


Problem 3 (ii)

In [10]:
y = weekly_ret.iloc[:,0].values
A = weekly_ret.iloc[:,1:].values
results = linear_solve_cholesky(y, A)
print(results)

[-0.00335278  0.7662948  -0.07776546  0.29795058  0.28117032 -0.05761416
 -0.41404465  0.14759435 -0.00880578]


## TODO Approximation Error

In [15]:
fit_value = get_fitted_vals(y,A)

approx_error1 = get_approx_error_banks(y, fit_value)
print("Approx Error for Q3(ii)")
print(approx_error1)

Approx Error for Q3(ii)
0.1294511977134714


Answer 3(iii) <br> The linear regression of the JPM returns with respect to the returns of the
other American financial companies, i.e., with respect to GS, MS, and BAC. What is
the approximation error of this linear regression?

In [16]:
A = weekly_ret[['GS', 'MS', 'BAC']].values
y = weekly_ret.iloc[:,0].values
results = linear_solve_cholesky(y, A)
print(results)
print()
fit_value = get_fitted_vals(y,A)

approx_error1 = get_approx_error_banks(y, fit_value)
print("Approx Error for Q3(iii)")
print(approx_error1)

[-0.00139422  0.65371049 -0.03750398  0.26128533]

Approx Error for Q3(iii)
0.1498840139544911


## TODO Approximation Error

Problem 3 (iv)

In [17]:
A = df.iloc[:,1:].values
y = df.iloc[:,0].values
results = linear_solve_cholesky(y, A)
print(results)

print()

fit_value = get_fitted_vals(y,A)

approx_error1 = get_approx_error_banks(y, fit_value)
print("Approx Error for Q3(iv)")
print(approx_error1)

[ 8.81655094  0.01660427  0.06681861  1.47131272  1.20743281  0.89990273
 -2.53732006  0.35021195 -0.17046126]

Approx Error for Q3(iv)
6.507914971443161
